# Data Preparation on the retail_store database

## First time setup

Setting up warehouses and creating and granting access to the retail_analyst role. 
Only need to run once:)

In [ ]:
use role accountadmin;

-- setup warehouse
create or replace warehouse retail_store_wh with warehouse_size='XSmall';
use warehouse retail_store_wh;

-- select database
use database ml;

-- set up retail role
create or replace role retail_analyst;
grant usage on database ml to role retail_analyst;
grant usage on schema ml.retail_store to role retail_analyst;
grant usage on warehouse retail_store_wh to role retail_analyst;
grant create table on schema ml.retail_store to role retail_analyst;
grant create view on schema ml.retail_store to role retail_analyst;

grant role retail_analyst to user mikkel;

# Before use

Make sure to run the cell below before use to select the correct database and schema.


In [ ]:
use database ml;
use schema ml.retail_store;

# Organising the data into new tables

Depending on what features I want to create I have orgnaised the data into tables that are easier to handle, for example, by giving products and transactions an id, and collecting transactions into checkouts. Read more below.



### Products
A table of products identified by a product id.

Schema:
- Product_id
- Product_category *
- Porduct_brand *
- Product_company_id *


Created from the transactions table using brand, company_id and category.



In [ ]:
create or replace table products as
    select 
        ROW_NUMBER() OVER(order by product_category, product_brand) as product_id,
        product_category,
        company_id,
        product_brand
    from transactions
        group by product_category, product_brand, company_id;


### Checkouts

The collection of all transactions by a customer on the same day at the same store (store_chain_id). 


- checkout_id
- customer_id
- chain_id
- date
- checkout amount

In [ ]:
create or replace table checkouts as
    select
            ROW_NUMBER() OVER(order by customer_id) as checkout_id,    
            t.customer_id,
            t.store_chain_id,
            t.date,
            sum(t.purchase_amount) as checkout_amount
    from    transactions t
    group by t.date, t.store_chain_id, t.customer_id;
select * from checkouts;
    

### transactions_main

New table for transactions where each is now identifiable by a transaction_id and holds a product_id and checkout_id for the tables created above.

Includes:  

- transaction_id
- checkout_id
- customer_id
- product_id
- product_grouping (categroy of the categories for each product_id)
- purchase_amount (price/cost)

In [ ]:
create or replace table transactions_main as
    select
            ROW_NUMBER() OVER(order by t.customer_id, t.product_grouping, t.purchase_amount) as transaction_id,
            c.checkout_id,
            t.customer_id,
            p.product_id,
            t.product_grouping,
            t.purchase_amount
    from    transactions t
    join    products p on
                p.product_category = t.product_category
                and p.product_brand = t.product_brand
                and p.company_id = t.company_id
    join    checkouts c on
                c.customer_id = t.customer_id
                and c.store_chain_id = t.store_chain_id
                and c.date = t.date;

select * from transactions_main limit 5;

# Final Table
model_data - the table of data that the models are to be trained and tested on.

- customer_id
- offer_product_id
- chain_cat_1
- chain_cat_2
- chain_cat_3
- previous_purchase_product_int
- previous_purchase_dept_int
- offer_value_1
- offer_value_2
- offer_value_3
- offer_value_4
- offer_value_5
- offer_value_6
- repeater_int (label column)


### Step 1: Joining the desired features from history, offers, and products into one table called: training_history table

history_main table:
- customer_id (for joining later)
- chain
- offer_product
    - Found in products table based on category, company, and brand
- offer_value
- offer_quantity
- repeater


In [ ]:
create or replace table history_main as 
    select
        h.customer_id, 
        h.store_chain_id,
        p.product_id as offer_product_id,
        o.offer_value,
        o.required_quantity as offer_quantity, 
        h.repeater
    from history h
    join offers o on h.offer_id = o.offer_id
    join products p on
        p.product_brand = o.brand_id 
        and p.company_id = o.company_id
        and p.product_category = o.product_category;

### Step 2: produce new features

Using transactions_main to create new features.


_Previous purchase product boolean_
- use transactions and customer_id in training_history table to see if the person has purchased the product previously.

_Previous purchase category boolean_
- use transactions and customer_id in training_history table to see if the person has purchased a product of the same category previously


Add both to training_history table

In [ ]:
create or replace view previous as
    select h.customer_id, h.offer_product_id
    from history_main h
    join transactions_main t on t.customer_id = h.customer_id
    where t.product_id = h.offer_product_id;

alter table history_main
add column previous_purchase boolean;

update history_main
    set previous_purchase = True
    where customer_id in (select customer_id from previous)
    and offer_product_id in (select offer_product_id from previous);

In [ ]:
create or replace view previous_cat as 
select distinct h.customer_id, h.offer_product_id
from history_main h
join transactions_main t on h.customer_id = t.customer_id
join transactions_main t2 on t2.product_id = h.offer_product_id
where t.product_grouping = t2.product_grouping;

alter table history_main
add column previous_purchase_category boolean;

update history_main
    set previous_purchase_category = True
    where customer_id in (select customer_id from previous_cat);


### Step 3: Select relevant columns into model_data table

In [ ]:
create or replace table model_data as
    select customer_id, offer_product_id, store_chain_id, offer_value, offer_quantity, previous_purchase, previous_purchase_category, repeater
    from history_main;

select * from model_data limit 5;

## Data Preprocessing on the training_table

### Checking for null values

In [ ]:
select
    count(case when store_chain_id is null then 1 end) as store_chain_id_count,
    count(case when offer_value is null then 1 end) as offer_value_count,
    count(case when offer_quantity is null then 1 end) as offer_quantity_count,
    count(case when previous_purchase is null then 1 end) as previous_purchase_count,
    count(case when previous_purchase_category is null then 1 end) as previous_purchase_category_count,
    count(case when repeater is null then 1 end) as repeater_count
from model_data;


### Make store_chain_id into three categories.
Store_chain_id makes up 160 categories which is too many for this model. Looking at a graph of the number of transactions per chain, I divide it into three categories. More than 10 million transactions, more than 2 million, and all else. 

In [ ]:
update model_data
set store_chain_id = (
    select 
        case
            when chain_count > 10000000 then 1
            when chain_count > 2000000 then 2
            else 3
        end
    from (
        select store_chain_id, count(*) as chain_count
        from transactions
        group by store_chain_id
        order by chain_count
    ) as chain_ordered
    where chain_ordered.store_chain_id = model_data.store_chain_id
);

Now one hot encoding the categories. And dropping store_chain_id.

In [ ]:
alter table model_data
    add chain_cat_1 integer;

alter table model_data
    add chain_cat_2 integer;

alter table model_data
    add chain_cat_3 integer;

update model_data
set
    chain_cat_1 = (
        case when store_chain_id = 1 then 1 else 0 end
    ),
    chain_cat_2 = (
        case when store_chain_id = 2 then 1 else 0 end
    ), 
    chain_cat_3 = (
        case when store_chain_id = 3 then 1 else 0 end
    );


alter table model_data
drop column store_chain_id;


### Drop Offer Quantity

Turns out offer quantity is always equal to 1, so dropping the column. 

In [ ]:
select * from model_data
where offer_quantity != 1;

In [ ]:
alter table model_data
drop column offer_quantity;

select * from model_data limit 5;

### One-hot-encooding offer value.

Offer-value takes on one out of 6 values so one-hot-encoding this to fit model requirements. 

In [ ]:
alter table model_data
add offer_value_1 integer;

alter table model_data
add offer_value_2 integer;

alter table model_data
add offer_value_3 integer;

alter table model_data
add offer_value_4 integer;

alter table model_data
add offer_value_5 integer;

alter table model_data
add offer_value_6 integer;

update model_data
set offer_value_1 = case when offer_value = 1.25 then 1 else 0 end;

update model_data
set offer_value_2 = case when offer_value = 3 then 1 else 0 end;

update model_data
set offer_value_3 = case when offer_value = 0.75 then 1 else 0 end;

update model_data
set offer_value_4 = case when offer_value = 1.5 then 1 else 0 end;

update model_data
set offer_value_5 = case when offer_value = 1 then 1 else 0 end;

update model_data
set offer_value_6 = case when offer_value = 2 then 1 else 0 end;


select * from model_data;

alter table model_data
drop column offer_value;

### Changing the datatype for previous_purchase, previous_purcahse_category, and reapeter

Due to how the data is transferred into df's, I need to replace the boolean columns with integers of 1, and 0. Seems strange, but necessary.

In [ ]:
alter table model_data
add previous_purchase_int integer;

alter table model_data
add previous_purchase_category_int integer;

alter table model_data
add repeater_int integer;

update model_data
set previous_purchase_int = case when previous_purchase = true then 1 else 0 end;

update model_data
set previous_purchase_category_int = case when previous_purchase_category = true then 1 else 0 end;

update model_data
set repeater_int = case when repeater = true then 1 else 0 end;

alter table model_data
drop column repeater;

alter table model_data
drop column previous_purchase;

alter table model_data
drop column previous_purchase_category;

## Final training_table


In [ ]:
select * from model_data limit 5;

## Add new feature, previous total purchase amount per customer_id

In [ ]:
create or replace view customer_total as
select customer_id, sum(purchase_amount) as total, from transactions_main
where purchase_amount > 0
group by customer_id
order by total;

alter table model_data 
add total float;

update model_data t
set total = coalesce(c.total, 0)
from customer_total c
where c.customer_id = t.customer_id;

In [ ]:
select * from model_data limit 5;